# GPT2-Fine-Tuning-With-LoRA
This notebook contains an experiment to fine-tune a GPT-2 model using LoRA (Low-Rank Adaptation) on a custom dataset. The goal is to adapt the pre-trained model to better suit specific tasks or domains by training it on a smaller, task-specific dataset.

In [1]:
import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    TrainingArguments, Trainer, 
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
from datasets import load_dataset, Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

---

As said before, we are going to use GPT-2, a transformer-based model developed by OpenAI. We chose this model because it can run on consumer hardware (RTX 4060 TI 16GB, in my case) and yet it is powerful enough to generate some intriguing results.

When you download a model using Hugging Face's `transformers` library, it will be saved on cahce. You can find the cache directory by running the following command:

```bash
huggingface-cli scan-cache
```

Models and datasets are stored in the `~/.cache/huggingface/` directory by default. To delete them, you can use the following command (or manually delete the files):

```bash
huggingface-cli delete-cache
```

When you download a model, you will also need a tokenizer. The tokenizer is responsible for converting text into a format that the model can understand, such as token IDs. The `transformers` library also provides pre-trained tokenizers that can be used with the models.

In [2]:
base_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2").to(device)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token

Bellow is a sample input text representing the kinf of task we are going to fine-tune the model on. Basically, the model must recieve a list of functions (tools) and their descriptions, allong with a user query.

In [3]:
system = """
SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "get_weather",
    "description": "Get the current weather",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to get the weather for"
            }
        },
        "required": [
            "city"
        ]
    }
}
"""

In [4]:
sample_tokens = tokenizer(
    "A tokenizer converts inputs into tokens that a model can understand."
)
sample_tokens

{'input_ids': [32, 11241, 7509, 26161, 17311, 656, 16326, 326, 257, 2746, 460, 1833, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
tokenizer.decode(sample_tokens['input_ids'])

'A tokenizer converts inputs into tokens that a model can understand.'

---

This is the function we are going to use to generate responses from the model. It follows the format {system} {user} {assistant} we defined earlier. The `system` part contains the list of functions and their descriptions, while the `user` part contains the user query. The `assistant` part is where the model will generate its response.

In [6]:
def ask(model, tokenizer, system, question):
    """Ask the model a question using the provided system prompt.

    Args:
        model: The language model to use for generating the response.
        tokenizer: The tokenizer to convert text to tokens.
        system: The system prompt that provides context for the model.
        question: The question to ask the model.

    Returns:
        str: The model's response to the question.
    """
    model.eval()
    prompt = f"{system}\n\n\nUSER: {question}\n\n\nASSISTANT:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id
    )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output


print(ask(base_model, tokenizer, system, question="What is the weather in New York?"))


SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "get_weather",
    "description": "Get the current weather",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to get the weather for"
            }
        },
        "required": [
            "city"
        ]
    }
}



USER: What is the weather in New York?


ASSISTANT: The weather in New York is the weather in New York.


SYSTEM: What is the weather in New York?


USER: What is the weather in New York?


ASSISTANT: The weather in New York is the weather in New York.


SYSTEM: What is the weather in New York?


USER: What is the weather in New York?


ASSISTANT: The weather in New York is the weather in New York.





As you can see, the model struggles to understand the task and generate a coherent response. The main reason is because GPT-2 is not a powerful model and is not good in generalizing across different tasks, especially when it comes to understanding inputs that were not in its original training dataset. This is where fine-tuning comes into play.

---

## Dataset
The glaiveai/glaive-function-calling-v2 dataset is a large-scale resource designed for training and evaluating language models on function calling and tool-use tasks. It contains over 112,000 examples, each structured with a system prompt that describes available functions (including their parameters and descriptions) and a chat transcript simulating realistic user-assistant interactions.

In each conversation, the assistant is expected to recognize when a function call is needed, generate the appropriate function call with correct arguments, and incorporate the function's response into a natural, helpful reply.


This dataset is ideal for developing and benchmarking models that need to reason about tool use, follow structured API schemas, and maintain coherent multi-turn dialogues involving external function calls.

In [7]:
dataset = load_dataset("glaiveai/glaive-function-calling-v2")
dataset

DatasetDict({
    train: Dataset({
        features: ['system', 'chat'],
        num_rows: 112960
    })
})

Here are some examples from the dataset.

In [8]:
dataset['train'][0]

{'system': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_exchange_rate",\n    "description": "Get the exchange rate between two currencies",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "base_currency": {\n                "type": "string",\n                "description": "The currency to convert from"\n            },\n            "target_currency": {\n                "type": "string",\n                "description": "The currency to convert to"\n            }\n        },\n        "required": [\n            "base_currency",\n            "target_currency"\n        ]\n    }\n}\n',
 'chat': "USER: Can you book a flight for me from New York to London?\n\n\nASSISTANT: I'm sorry, but I don't have the capability to book flights. My current function allows me to get the exchange rate between two currencies. If you need help with that, feel free to ask! <|endoftext|>\n\n\n"}

In [9]:
dataset['train'][1]

{'system': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_news_headlines",\n    "description": "Get the latest news headlines",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "country": {\n                "type": "string",\n                "description": "The country for which to fetch news"\n            }\n        },\n        "required": [\n            "country"\n        ]\n    }\n}\n',
 'chat': 'USER: Can you tell me the latest news headlines for the United States?\n\n\nASSISTANT: <functioncall> {"name": "get_news_headlines", "arguments": \'{"country": "United States"}\'} <|endoftext|>\n\n\nFUNCTION RESPONSE: {"headlines": ["Biden announces new vaccine mandates", "Hurricane Ida devastates Louisiana", "Apple unveils new iPhone", "NASA\'s Perseverance rover collects first Mars rock sample"]}\n\n\nASSISTANT: Here are the latest news headlines for the United States:\n1. Biden 

As you can see, the dataset also contains data from more than one round of conversation. However, we are going to modify it to only include one round per sample, in order to simplify the fine-tuning process and model understanding.

In [10]:
def process_chat(system, chat):
    """Process a chat conversation into training examples. Each sample will consist of a single round of conversation.

    Args:
        system (str): The system prompt.
        chat (str): The chat conversation as a string.

    Returns:
        list: A list of dictionaries containing context and target for training.
    """
    blocks = chat.strip().split('\n\n')
    parsed_blocks = []
    for block in blocks:
        block = block.strip()
        if block.startswith('USER:'):
            parsed_blocks.append(('user', block))
        elif block.startswith('ASSISTANT:'):
            parsed_blocks.append(('assistant', block))
        elif block.startswith('FUNCTION RESPONSE:'):
            parsed_blocks.append(('function_response', block))
    
    training_examples = []
    history = [system]
    for speaker, block in parsed_blocks:
        if speaker == 'assistant':
            # Extract assistant's content after "ASSISTANT: "
            content = block[len('ASSISTANT:'):].strip()
            # Context is all previous messages plus "ASSISTANT: "
            context = '\n'.join(history) + '\nASSISTANT: '
            target = content
            training_examples.append({'context': context, 'target': target})
            # Add this block to history
            history.append(block)
        else:
            history.append(block)
    return training_examples


processed_data = []
for example in dataset['train']:
    system = example['system']
    chat = example['chat']
    processed_examples = process_chat(system, chat)
    processed_data.extend(processed_examples)

processed_data[0]

{'context': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_exchange_rate",\n    "description": "Get the exchange rate between two currencies",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "base_currency": {\n                "type": "string",\n                "description": "The currency to convert from"\n            },\n            "target_currency": {\n                "type": "string",\n                "description": "The currency to convert to"\n            }\n        },\n        "required": [\n            "base_currency",\n            "target_currency"\n        ]\n    }\n}\n\nUSER: Can you book a flight for me from New York to London?\nASSISTANT: ',
 'target': "I'm sorry, but I don't have the capability to book flights. My current function allows me to get the exchange rate between two currencies. If you need help with that, feel free to ask! <|endoftext|>"}

In [11]:
processed_data[1]

{'context': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_news_headlines",\n    "description": "Get the latest news headlines",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "country": {\n                "type": "string",\n                "description": "The country for which to fetch news"\n            }\n        },\n        "required": [\n            "country"\n        ]\n    }\n}\n\nUSER: Can you tell me the latest news headlines for the United States?\nASSISTANT: ',
 'target': '<functioncall> {"name": "get_news_headlines", "arguments": \'{"country": "United States"}\'} <|endoftext|>'}

Comparing the example [1], there is only one response at a time. With this modification, the dataset size got 3 times bigger. It is worth mentioning that, before the implementation of this notebook, I tried to fine-tune the model using the original dataset, but it was not able to generate coherent responses. After modifying the dataset, the quality of the generated responses improved significantly, as you can check at the end of this notebook.

In [12]:
train_dataset = Dataset.from_list(processed_data)
train_dataset

Dataset({
    features: ['context', 'target'],
    num_rows: 356860
})

In [13]:
del dataset
del processed_data

The function below prepares the dataset for fine-tuning by tokeniznng it and defining the input and output formats. For the sake of RAM usage and training speed, we are going to use a small subset of the dataset (100,000 samples) for fine-tuning. You can increase this number if you have more RAM and want to train the model with more data.


In [14]:
def tokenize_dataset(examples):
    """Tokenize the dataset for training.

    Args:
        examples (dict): A dictionary containing 'context' and 'target' keys.

    Returns:
        dict: A dictionary with tokenized inputs and labels.
    """
    full_texts = [context + target for context, target in zip(examples['context'], examples['target'])]
    tokenized = tokenizer(full_texts, padding="max_length", truncation=True, max_length=1024, return_offsets_mapping=True)
    
    labels = []
    for i in range(len(tokenized["input_ids"])):
        context_len = len(examples['context'][i])
        offset_mapping = tokenized["offset_mapping"][i]
        label = [-100] * len(tokenized["input_ids"][i])
        for j, (start, end) in enumerate(offset_mapping):
            if start >= context_len:
                label[j] = tokenized["input_ids"][i][j]
        labels.append(label)
    
    tokenized["labels"] = labels
    return tokenized


# writer_batch_size=1 cache_file_name='test'
tokenized_train_dataset = train_dataset.select(range(100_000)).map(tokenize_dataset, batched=True)
tokenized_train_dataset

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'target', 'input_ids', 'attention_mask', 'offset_mapping', 'labels'],
    num_rows: 100000
})

In [15]:
del train_dataset

You can see below the our dataset now consists of 'context', 'target', 'input_ids', 'attention_mask', 'offset_mapping', 'labels'. The trainig phase will not use the 'context' and 'target' columns, but they are useful for debugging and understanding the dataset.

In [16]:
tokenized_train_dataset[0]

{'context': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_exchange_rate",\n    "description": "Get the exchange rate between two currencies",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "base_currency": {\n                "type": "string",\n                "description": "The currency to convert from"\n            },\n            "target_currency": {\n                "type": "string",\n                "description": "The currency to convert to"\n            }\n        },\n        "required": [\n            "base_currency",\n            "target_currency"\n        ]\n    }\n}\n\nUSER: Can you book a flight for me from New York to London?\nASSISTANT: ',
 'target': "I'm sorry, but I don't have the capability to book flights. My current function allows me to get the exchange rate between two currencies. If you need help with that, feel free to ask! <|endoftext|>",
 'input_ids

In [17]:
tokenized_train_dataset[1]

{'context': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_news_headlines",\n    "description": "Get the latest news headlines",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "country": {\n                "type": "string",\n                "description": "The country for which to fetch news"\n            }\n        },\n        "required": [\n            "country"\n        ]\n    }\n}\n\nUSER: Can you tell me the latest news headlines for the United States?\nASSISTANT: ',
 'target': '<functioncall> {"name": "get_news_headlines", "arguments": \'{"country": "United States"}\'} <|endoftext|>',
 'input_ids': [23060,
  25361,
  25,
  921,
  389,
  257,
  7613,
  8796,
  351,
  1895,
  284,
  262,
  1708,
  5499,
  13,
  5765,
  606,
  611,
  2672,
  532,
  198,
  90,
  198,
  220,
  220,
  220,
  366,
  3672,
  1298,
  366,
  1136,
  62,
  10827,
  62,
  2256,
  6615,
  1600,
  1

---

## LoRA
LoRA is a technique that allows for efficient fine-tuning of large language models by introducing low-rank adaptations to the model's weights. This approach reduces the number of trainable parameters, making it feasible to fine-tune large models on smaller datasets without requiring extensive computational resources.

The first setp of LoRA utilization is to adapt a pre-trained model by adding low-rank matrices to the existing weights.

In [18]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    fan_in_fan_out=True,
)

lora_model = get_peft_model(base_model, lora_config)

Below you can see that instead of training the entire model (124,734,720 parametes), we are only training the LoRA layers (294,912 parameters). This significantly reduces the number of trainable parameters and speeds up the training process.

In [19]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


Finally, we can use the `Trainer` class from the `transformers` library to train the model. The `Trainer` class provides a high-level API for training and evaluating models, making it easy to fine-tune our GPT-2 model with LoRA. We decided to train for just 1 epoch, but you can increase this number if you want to train the model for more epochs. The training process may take several minutes, depending on your hardware and the size of the dataset.

In [20]:
training_args = TrainingArguments(
    output_dir="./gpt2-function-calling-lora",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    num_train_epochs=1,
    logging_steps=100,
    save_strategy="epoch",
    fp16=True,
    push_to_hub=False,
    optim="adamw_torch",
    label_names=["labels"]
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train_dataset.shuffle(seed=9),
    processing_class=tokenizer,
    data_collator=data_collator
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.924900
200,1.582200
300,1.350000
400,1.175300
500,1.080600
600,1.124600
700,1.069700
800,1.010400
900,1.034800
1000,1.025100


TrainOutput(global_step=12500, training_loss=0.9090988269042969, metrics={'train_runtime': 7634.1618, 'train_samples_per_second': 13.099, 'train_steps_per_second': 1.637, 'total_flos': 5.24396003328e+16, 'train_loss': 0.9090988269042969, 'epoch': 1.0})

We can now save the fine-tuned model.

In [21]:
trainer.save_model("./gpt2-function-calling-lora")
tokenizer.save_pretrained("./gpt2-function-calling-lora")

('./gpt2-function-calling-lora\\tokenizer_config.json',
 './gpt2-function-calling-lora\\special_tokens_map.json',
 './gpt2-function-calling-lora\\vocab.json',
 './gpt2-function-calling-lora\\merges.txt',
 './gpt2-function-calling-lora\\added_tokens.json',
 './gpt2-function-calling-lora\\tokenizer.json')

---

In [22]:
system = """
SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "get_csv_shape",
    "description": "Get the shape of a CSV file",
    "parameters": {
        "type": "object",
        "properties": {
            "csv_file": {
                "type": "string",
                "description": "The path to the CSV file"
            }
        },
        "required": [
            "csv_file"
        ]
    }
}
"""

In [24]:
print(ask(lora_model, tokenizer, system, question="What is the shape of the CSV file at /path/to/file.csv?"))


SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "get_csv_shape",
    "description": "Get the shape of a CSV file",
    "parameters": {
        "type": "object",
        "properties": {
            "csv_file": {
                "type": "string",
                "description": "The path to the CSV file"
            }
        },
        "required": [
            "csv_file"
        ]
    }
}



USER: What is the shape of the CSV file at /path/to/file.csv?


ASSISTANT: <functioncall> {"name": "get_csv_shape", "arguments": '{"csv_file": "path/to/file.csv"}'} } 
FUNCTION RESPONSE: {"shape": "file", "file_name": "file_file.csv"}
ASSISTANT: The shape of the CSV file at /path/to/file.csv is: file_name: file_file.csv. 
ASSIST


Asking the model to generate a response using the same we tried before fine-tuning, we can see that the model is now able to generate a much more coherent response. The model has learned to understand the task and generate a response that is relevant to the user query.

In [27]:
print(ask(lora_model, tokenizer, system, question="What is the weather in New York?"))


SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "get_csv_shape",
    "description": "Get the shape of a CSV file",
    "parameters": {
        "type": "object",
        "properties": {
            "csv_file": {
                "type": "string",
                "description": "The path to the CSV file"
            }
        },
        "required": [
            "csv_file"
        ]
    }
}



USER: What is the weather in New York?


ASSISTANT: <functioncall> {"name": "get_csv_shape", "arguments": '{"csv_file": "New York", "path": "https://data.csv.com/weather"}'} } 
FUNCTION RESPONSE: {"shape": "summed"}
ASSISTANT: The weather in New York is in the current state of New York. 
ASSISTANT: The weather in New York is in the current state of New York


Of course, it is not even close to perfect, but it is a significant improvement over the original model. GPT-2 is not a powerful model and was never experienced in this kind of task. Also remember that we only trained the model for 1 epoch and using a fraction of the dataset, so there is still room for improvement. You can try training the model for more epochs or using a larger dataset to see if the quality of the generated responses improves further.

---

https://huggingface.co/docs/transformers/quicktour

https://huggingface.co/docs/peft/quicktour

https://huggingface.co/docs/datasets/quickstart

https://huggingface.co/openai-community/gpt2